In [1]:
import pandas as pd
import numpy as np
import json
import gzip
import os
from os import listdir
import seaborn as sns
import matplotlib.pyplot as plt

In [19]:
#read table to dataframe and create train/test split with equal distribution of labels within table
#example small book
small = pd.read_csv('../../src/data/schemafiltereddata/small_tables.csv')
small_books = small[small['Class']=='Book']
small_books

,filename,Class,num_rows,num_columns,num_columns_selected,num_NAs,percentage_col_NA,percentage_table_NA,sum_NA_sel_col,Book_aggregaterating,Book_author,Book_bookedition,Book_bookformat,Book_datepublished,Book_genre,Book_inlanguage,Book_isbn,Book_name,Book_numberofpages,Book_offers,Book_publisher,CreativeWork_aggregaterating,CreativeWork_alternativeheadline,CreativeWork_articlebody,CreativeWork_author,CreativeWork_commentcount,CreativeWork_copyrightholder,CreativeWork_copyrightyear,CreativeWork_creator,CreativeWork_datecreated,CreativeWork_datemodified,CreativeWork_datepublished,CreativeWork_genre,CreativeWork_headline,CreativeWork_inlanguage,CreativeWork_interactioncount,CreativeWork_interactionstatistic,CreativeWork_interactiontype,CreativeWork_keywords,CreativeWork_mainentityofpage,CreativeWork_name,CreativeWork_offers,CreativeWork_publisher,CreativeWork_version,Event_doortime,Event_duration,Event_enddate,Event_eventattendancemode,Event_eventstatus,Event_location,Event_name,Event_organizer,Event_performer,Event_performers,Event_typicalagerange,Hotel_address,Hotel_aggregaterating,Hotel_email,Hotel_geo,Hotel_name,Hotel_pricerange,Hotel_starrating,Hotel_telephone,LocalBusiness_address,LocalBusiness_addresslocality,LocalBusiness_addressregion,LocalBusiness_aggregaterating,LocalBusiness_areaserved,LocalBusiness_citystatezip,LocalBusiness_contactpoint,LocalBusiness_email,LocalBusiness_faxnumber,LocalBusiness_founder,LocalBusiness_geo,LocalBusiness_hasmap,LocalBusiness_legalname,LocalBusiness_location,LocalBusiness_name,LocalBusiness_openinghours,LocalBusiness_openinghoursspecification,LocalBusiness_owns,LocalBusiness_paymentaccepted,LocalBusiness_postalcode,LocalBusiness_pricerange,LocalBusiness_review,LocalBusiness_sameas,LocalBusiness_streetaddress,LocalBusiness_telephone,MusicAlbum_byartist,MusicAlbum_name,MusicAlbum_numtracks,MusicAlbum_track,MusicRecording_byartist,MusicRecording_datecreated,MusicRecording_datepublished,MusicRecording_duration,MusicRecording_genre,MusicRecording_inalbum,MusicRecording_name,Person_additionalname,Person_address,Person_affiliation,Person_birthdate,Person_birthplace,Person_contactpoint,Person_deathdate,Person_email,Person_familyname,Person_faxnumber,Person_gender,Person_givenname,Person_height,Person_homelocation,Person_jobtitle,Person_knowslanguage,Person_mainentityofpage,Person_makesoffer,Person_memberof,Person_nationality,Person_sameas,Person_telephone,Person_weight,Person_worklocation,Person_worksfor,Place_geo,Place_name,Place_openinghoursspecification,Place_sameas,Place_telephone,Product_additionalproperty,Product_address,Product_aggregaterating,Product_alternatename,Product_audience,Product_author,Product_availability,Product_bestrating,Product_brand,Product_breadcrumb,Product_category,Product_color,Product_condition,Product_datepublished,Product_depth,Product_disambiguatingdescription,Product_geo,Product_gtin,Product_gtin12,Product_gtin13,Product_gtin14,Product_gtin8,Product_height,Product_identifier,Product_isrelatedto,Product_issimilarto,Product_itemcondition,Product_itemlistelement,Product_mainentityofpage,Product_manufacturer,Product_material,Product_model,Product_mpn,Product_name,Product_offerdetails,Product_offers,Product_price,Product_pricecurrency,Product_ratingvalue,Product_releasedate,Product_review,Product_reviewcount,Product_reviews,Product_shop-currency,Product_title,Product_weight,Product_width,Product_worstrating,Recipe_aggregaterating,Recipe_author,Recipe_cookingmethod,Recipe_cooktime,Recipe_datemodified,Recipe_datepublished,Recipe_headline,Recipe_ingredients,Recipe_ispartof,Recipe_keywords,Recipe_mainentityofpage,Recipe_name,Recipe_nutrition,Recipe_performtime,Recipe_preptime,Recipe_publisher,Recipe_recipecategory,Recipe_recipecuisine,Recipe_recipeingredient,Recipe_recipeinstructions,Recipe_recipeyield,Recipe_review,Recipe_suitablefordiet,Recipe_totaltime,Restaurant_acceptsreservations,Restaurant_address,Restaurant_aggregaterating,Restaurant_geo,Restaurant_hasmenu,Restaura

In [21]:
from skmultilearn.model_selection import iterative_train_test_split

In [31]:
path='/work-ceph/bizer-tp2021/data_integration_using_deep_learning/src/data/schemafiltereddata/Book/'

In [32]:
test_books = pd.read_json(path+'Book_aardvark-books.com_September2020.json.gz', compression='gzip', lines=True)
test_books

,row_id,name,author,publisher,isbn,bookformat,datepublished,offers,page_url
0,0,Pacific Steam The British Pacific Locomotive,"{'name': 'Evans, Martin'}",{'name': 'Model Aeronautical Press'},None,Hardcover,1960-01-01,"{'pricevaliduntil': '2020-10-11', 'pricecurren...",https://www.aardvark-books.com/shop/other-non-...
1,1,Early Aviation In North Wales,"{'name': 'Sloan, Roy'}",{'name': 'Gwasg Carreg Gwalch'},0863811191,Paperback,1989-05-01,"{'pricevaliduntil': '2020-10-11', 'pricecurren...",https://www.aardvark-books.com/shop/other-non-...
2,2,Cessna 150 Pilots Guide: A Pilot's Guide (The ...,"{'name': 'Pratt, Jeremy M.'}",{'name': 'Airplan Flight Equipment Ltd'},187478325X,Paperback,1993-07-01,"{'pricevaliduntil': '2020-10-11', 'pricecurren...",https://www.aardvark-books.com/shop/other-non-...
3,3,Early MG,{'name': 'Jennings P L'},{'name': 'P. L. Jennings'},0951450905,Hardcover,1989-01-01,"{'pricevaliduntil': '2020-10-11', 'pricecurren...",https://www.aardvark-books.com/shop/other-non-...
4,4,"Selected Poems, 1908-59","{'name': 'Pound, Ezra'}",{'name': 'Faber & Faber'},None,Hardcover,1975-10-27,"{'pricevaliduntil': '2020-10-11', 'pricecurren...",https://www.aardvark-books.com/shop/rarebooks/...
5,5,Steaming Through Britain: Complete Guide to Br...,{'name': 'Chris Ellis'},{'name': 'Conway Maritime Press Ltd'},184486121X,Hardcover,2010-10-12,"{'pricevaliduntil': '2020-10-11', 'pricecurren...",https://www.aardvark-books.com/shop/other-non-...
6,6,Cruising: A Manual for Small Cruiser Sailing,"{'name': 'Sleightholme, J.D.'}",{'name': 'Adlard Coles Nautical'},0713634774,Paperback,1990-09-03,"{'pricevaliduntil': '2020-10-11', 'pricecurren...",https://www.aardvark-books.com/shop/other-non-...
7,7,The Trains We Loved,"{'name': 'Whitehouse, Patrick, St.John Thomas,...",{'name': 'David & Charles'},None,Hardcover,1994-09-30,"{'pricevaliduntil': '2020-10-11', 'pricecurren...",https://www.aardvark-books.com/shop/other-non-...
8,8,Steam Portfolios: North Eastern Steam Bk. 5,"{'name': 'Johnson, M.'}",{'name': 'Ian Allan Publishing'},0711018693,Hardcover,1989-11-01,"{'pricevaliduntil': '2020-10-11', 'pricecurren...",https://www.aardvark-books.com/shop/other-non-...
9,9,GREAT WESTERN: ALBUM NO.2.,"{'name': 'Riley, R.C.'}",{'name': 'Ian Allen'},0711000735,Hardcover,1971-01-01,"{'pricevaliduntil': '2020-10-11', 'pricecurren...",https://www.aardvark-books.com/shop/other-non-...


In [40]:
X_train, y_train, X_test, y_test = iterative_train_test_split(test_books, test_size = 0.2)


TypeError: iterative_train_test_split() missing 1 required positional argument: 'y'